In [1]:
import pandas as pd
import re
from ast import literal_eval
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '90'
formatted_attribute = 'height'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'
field1='Height [Nom]'
fields = [field1]

In [3]:
df_cur, df_custom=parameters(customer_id, formatted_attribute, field1)
df_custom=custom_field(df_custom,field1,formatted_attribute)
matchcust=df_custom[['external_id',curation_col]]
print(len(matchcust))
rounding(df_custom,curation_col)

1692
16


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, Height [Nom], match, Q:height, rounding]
Index: []

In [4]:
print(len(df_cur))

1676


# General Regex

In [5]:
regex_pattern_length=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3Le?n?g?t?h?\b)|()'''
df_cur['matches_length'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'''(?i)((?<!up.to.)(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}He?i?g?h?t?(?! Adjustable)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Wi?d?t?h?\b)|()'''
df_cur['matches_width'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'''(?i)((?<!\d)(?<!\w )(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Diam?e?t?e?r?\b)|()'''
df_cur['matches_diameter'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df_cur['matches_ft'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d'                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# df_cur['new_matches'] = df_cur['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'''(?i)((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inch|foot|fe?e?t|”)?\s?(?:[LWHD]|diam)?\s?x.?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inch|foot|fe?e?t|”)?\s?(?:[LWHD]|diam)?\s?x?.?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)?\W{0,3}(?:"|'|''|inch|foot|fe?e?t|”)?\s?(?:[LWHD]|diam)?\s?)|(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inch|foot|fe?e?t|”)\s?(?:[LWHD]|diam)?\s?|()'''
df_cur['blank_matches'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_matches_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

# Height

In [6]:
from natsort import natsorted
regex_pattern=r'''(?i)((?<!\-)(?<!\d)\d+(?:\.\d+|.\d+\/\d+)?[^\w\d]{0,3}He?i?g?h?t?(?!\sLabel)(?!.Adjustment)(?!.Adjustable)\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)(?<!\-)\d+(?!\.\d)(?<!\d\\\"\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!Adjustable)(?!\sLabel)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''           
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

# Filter out the SKUs that dont have numbers
pat_number=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(pat_number, str(x)))
height_new=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]
del height_new['new_matches']

height_new['matches']=height_new['matches'].apply(lambda x: natsorted(x)).apply(lambda x: sorted(x))
height_new['matches']=height_new['matches'].apply(lambda x: re.sub(r"(?<=\d)\'",' ft',str(x))).apply(lambda x: re.sub(r'(?i)height|"\s?high',' in',str(x))).apply(lambda x: re.sub(r'(?i)height|\\" H|"H|H',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"|�|•','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
height_new['matchez']=height_new['matches'].apply(lambda x: re.sub(r"\'",' ft',str(x))).apply(lambda x: re.sub(r'ft in','ft',str(x)))


curate(height_new, 'matchez', formatted_attribute)
print(len(height_new))
matchesnew=height_new[['external_id',curation_col]]
matchesnew[curation_col].explode().value_counts()
rounding(height_new, 'matchez')

17


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, Height [Nom], matches_height, blank_matches, blank_matches_long, matches, matchez, Q:height, rounding]
Index: []

# N/A Values

In [7]:
regex_pattern_na=r'''(?i)((?<!Cord Measures.{10})\d+\.\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})\d+.?\d+\/\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.\d).{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.).{0,2}['"].{0,2}[LWHD]?)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|(measure)|()'''
regex_pattern_measure=r'(?i)(measure)|()'
df_na=df_cur
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na['na_measure_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_measure, x))

df_na=df_na[(df_na['matches_height'].astype(str)=='[]')&(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')&(df_na['blank_matches_long'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))
# matchesna

244


In [8]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnew))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcust))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

matchesnew
matchcust
matchesna


In [ ]:
stop

# send to the folder for upload

In [9]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesnew)

In [10]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchcust)

In [11]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesna)